In [585]:
import requests
import random
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime , timedelta
import re
import os



path = "//Users//hajiaga//Desktop//DataScience//chromedriver"

driver = webdriver.Chrome()

driver.implicitly_wait(5)


In [586]:
def azetoengdate(string):
    if string == "Bugün":
        today = datetime.today()

        return today.strftime('%m %d')
        
    elif string == "Dünən":

        yesterday = datetime.today() - timedelta(days=1)

        return yesterday.strftime('%m %d')
    else:
        gun = string[:2]

        ay = str(re.findall(r'\D+', string)[0]).replace(" ","").lower()

        if ay == "yanvar":
            numay = "01"
        elif ay == "fevral":
            numay = "02"
        elif ay == "mart":
            numay = "03"
        elif ay == "aprel":
            numay = "04"
        elif ay == "may":
            numay = "05"
        elif ay == "iyun":
            numay = "06"
        elif ay == "iyul":
            numay = "07"
        elif ay == "avqust":
            numay = "08"
        elif ay == "sentyabr":
            numay = "09"
        elif ay == "oktyabr":
            numay = "10"
        elif ay == "noyabr":
            numay = "11"
        elif ay == "dekabr":
            numay = "12"
        else:
            numay ="Nan"
        
        return f"{numay} {gun}"


In [587]:
def scrape_new_window(href , dic):
    driver.get(href)

    dic["full_name"].append((driver.find_element(By.XPATH , "//h1[@class = 'product-title']")).text)

    temp = driver.find_element(By.XPATH , "//div[@class = 'product-properties__column']")

    dic["location"].append((temp.find_element(By.XPATH , "//span[@class = 'product-properties__i-value']")).text)

    comp_name = "other" if (temp.find_element(By.TAG_NAME , "a")).text == "Digər" else (temp.find_element(By.TAG_NAME , "a")).text
    
    dic["comp_name"].append(comp_name)

    tempp = driver.find_element(By.XPATH , "//div[@class = 'product-price__i product-price__i--bold']")
    
    temppspans = tempp.find_elements(By.TAG_NAME , "span")

    dic["price"].append(temppspans[0].text)

    dic["currency"].append(temppspans[1].text)

    temp1 = driver.find_element(By.XPATH , "//div[@class = 'product-info product-info__statistics']")

    temp2 = temp1.find_elements(By.TAG_NAME , "div")

    date = (str(temp2[1].find_element(By.TAG_NAME , "span").text)).split(',')[0]

    temp = [1 if "product-owner" in div.get_attribute("class") else 0 for div in driver.find_elements(By.CSS_SELECTOR , "#js-lot-page > div > aside > div > div > div")]

    dic["type"].append('individual_seller' if sum(temp) == 1 else 'shop')
    
    dic["date"].append(azetoengdate(date))

    try:
        desc = driver.find_element(By.XPATH , "//*[@id='js-lot-page']/div/main/section[3]/div/div/p").text
    except:
        desc = 'Nan'

    dic["description"].append(desc)

    driver.back()

    return dic


In [588]:
path1 = "https:/tap.az"

driver.get(path1)

time.sleep(5)
min_price = 500

max_price = 2500

# categorya "//a[@data-for = 'consumer-electronics']"



try:
    category = driver.find_element(By.XPATH , "//a[@data-for = 'consumer-electronics']")

    driver.get(category.get_attribute('href'))

    dropdown = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'price']")

    dropdown.click()
    
    minprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_lower']")

    maxprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_upper']")

    minprice.send_keys(f"{min_price}")

    maxprice.send_keys(f"{max_price}")

    time.sleep(5)

    categories = driver.find_element(By.XPATH , "//div[@class = 'subcategories-inner']")

    uls = categories.find_elements(By.TAG_NAME , 'ul')

    for ul in uls:
        
        lis = ul.find_elements(By.TAG_NAME , 'li')

        for li in lis:

            if "noutbuk" in str(li.find_element(By.TAG_NAME , 'a').get_attribute('href')).lower():

                href = li.find_element(By.TAG_NAME , 'a').get_attribute('href')

                break
    
    driver.get(href)

    dropdown_new = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'new']")

    dropdown_new.click()

    dropdown_new.find_element(By.TAG_NAME,"ul").find_elements(By.TAG_NAME , 'li')[1].click()
    
    time.sleep(3)

    dic_comp = {}

    dic_comp["location"] = []

    dic_comp["date"] = []

    dic_comp["link"] = []

    dic_comp["price"] = []

    dic_comp["comp_name"] = []

    dic_comp["full_name"] = []

    dic_comp["currency"] = []

    dic_comp["description"] = []

    dic_comp["type"] = []

    last_length = 0

    count_= str(href).count("/")+1

    while True:

        products = driver.find_elements(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")

        if len(products) == last_length:
            break
       
        i = 0

        for product in products:
            
            i+=1

            if i>last_length:
                
                if ("pagination" not in product.get_attribute("class")):
                   
                    a = product.find_element(By.TAG_NAME , "a")

                    if (str(a.get_attribute("href")).count('/') == count_):

                        dic_comp["link"].append(a.get_attribute("href"))

                        dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)
    
        last_length = len(products)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(10)





    # The final problem is that i count until 48 but after that website is getting refreshed so i need to make a function for that and stop it gettinf the same thing after 48
            

    #Ilk usula don her defe scroll ele yeniden tap butun seyleri biraz vaxt alacaq
        


    
        
finally:
    driver.close()



In [592]:
print(len(dic_comp['link']))

1578


In [591]:
for i in dic_comp.keys():
    print(len(dic_comp[i]))
dftapaz = pd.DataFrame.from_dict(dic_comp)

dftapaz.drop([''])

filename = "Users/hajiaga/Desktop/DataScience/tapazdata2.csv"
filename2 = "Users/hajiaga/Desktop/DataScience/tapazdata2.xlsx"
if os.path.exists(filename):
    os.remove(filename)
if os.path.exists(filename2):
    os.remove(filename2)
dftapaz.to_csv(filename, index=False) 
dftapaz.to_excel(filename2, index=False) 

dftapaz

1578
1578
1578
1578
1578
1578
1578
1578
1578


,location,date,link,price,comp_name,full_name,currency,description,type
0,Bakı,04 02,https://tap.az/elanlar/elektronika/noutbuklar/...,629,HP,"Noutbuk ""HP 250G8 Silver""",AZN,__𝐙𝐞𝐦𝐚𝐧𝐞𝐭𝐥𝐢 𝐍𝐨𝐭𝐞𝐛𝐨𝐨𝐤𝐥𝐚𝐫𝐢𝐧 𝐓𝐞𝐤 𝐔𝐧𝐯𝐚𝐧𝐢__\n___Bir...,shop
1,Bakı,04 02,https://tap.az/elanlar/elektronika/noutbuklar/...,670,Lenovo,Lenovo Thinkpad T580,AZN,YENİ İL İLƏ BAĞLI XÜSUSİ ENDİRİMLƏR!!!\nİNDİBÜ...,shop
2,Bakı,04 02,https://tap.az/elanlar/elektronika/noutbuklar/...,550,Dell,"Noutbuk ""Dell Latitude 5480""",AZN,Dell Latitude 5480 laptop\nCore i5 - 7300u 2.6...,individual_seller
3,Bakı,04 02,https://tap.az/elanlar/elektronika/noutbuklar/...,1 349,Lenovo,"Noutbuk ""Lenovo Legion""",AZN,__𝐙𝐞𝐦𝐚𝐧𝐞𝐭𝐥𝐢 𝐍𝐨𝐭𝐞𝐛𝐨𝐨𝐤𝐥𝐚𝐫𝐢𝐧 𝐓𝐞𝐤 𝐔𝐧𝐯𝐚𝐧𝐢__\n___Bir...,shop
4,Bakı,04 02,https://tap.az/elanlar/elektronika/noutbuklar/...,1 899,Asus,Asus Tuf Gaming,AZN,_𝐙𝐞𝐦𝐚𝐧𝐞𝐭𝐥𝐢 𝐍𝐨𝐭𝐞𝐛𝐨𝐨𝐤𝐥𝐚𝐫𝐢𝐧 𝐓𝐞𝐤 𝐔𝐧𝐯𝐚𝐧𝐢__\n___Bir ...,shop
...,...,...,...,...,...,...,...,...,...
1573,Bakı,03 06,https://tap.az/elanlar/elektronika/noutbuklar/...,960,HP,"Noutbuk ""HP 17""",AZN,"Ram 20 gb, ssd 128 gb, window's 11.",individual_seller
1574,Bakı,03 31,https://tap.az/elanlar/elektronika/noutbuklar/...,1 250,Asus,"Noutbuk ""Asus Rog Strix""",AZN,🔥Asus Rog Strix 🔥(120Hz Display)\n🔹İntel Core ...,shop
1575,Bakı,03 31,https://tap.az/elanlar/elektronika/noutbuklar/...,500,Dell,"Noutbuk ""Dell Latitude""",AZN,🔥Dell Latitude E7450🔥(Business Model)\n🔹İntel ...,shop
1576,Bakı,03 31,https://tap.az/elanlar/elektronika/noutbuklar/...,549,HP,HP ProBook G3,AZN,🔥Hp ProBook G3🔥(Business Model)\n🔹İntel Core i...,shop
